In [ ]:
import numpy as np, pandas as pd 
pd.set_option('display.max_columns', None)
import sqlite3, sqlalchemy, sys, os  

sys.path.append(os.path.abspath(os.path.join(os.path.dirname('functions.py'), '..', '..', 'PY')))
import functions  

seasons = functions.generate_seasons(1973, 2024)
# seasons = functions.generate_seasons(2007, 2024)

In [ ]:
engine = sqlalchemy.create_engine("sqlite:///../../DB/ballbase.db") 
def clean_player(name): 
    i = name.find('*')   
    name = name[:i] if i > -1 else name 
    return name 
for season in seasons:
    dir = f"../../static/CSV/{season}/" 

    names = ['pergame_'+season, 'advanced_'+season, 'per100_'+season, 'playbyplay_'+season, 'totals_'+season]  
    dfs = {} 

    invalid = ['-9999', 'Unnamed', 'Rk', 'Player-additional']

    for name in names:
        dfs[name] = pd.read_csv(dir+name+'.csv') 
        for inv in invalid:
            for col in dfs[name].columns:
                if inv in col or col in invalid:

                    dfs[name] = dfs[name].drop([col], axis=1)
                    
    dfs['totals_'+season] = dfs['totals_'+season].loc[:,['PTS', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'MP', 'FG', 'FGA', '3PA', '2P', '2PA', 'FT', 'FTA', 'ORB', 'DRB']] 
    dfs['totals_'+season].columns = 'T' + dfs['totals_'+season].columns 
    dfs['totals_'+season]['Player'] =  dfs['pergame_'+season]['Player']

    def combine_dfs(dfs, names):
        curr = None
        for name in names:
            if curr is None: 
                curr = dfs[name]
            else:
                diff = dfs[name].columns.difference(curr.columns).tolist()
                diff.append('Player') 
                curr = pd.merge(curr, dfs[name].loc[:,diff], on = 'Player') 
        return curr     

    master = combine_dfs(dfs, names) 
    master['Player'] = master['Player'].apply(lambda x: clean_player(x)) 

    for name in names: 
        name_temp = name 
        if name == 'pergame_'+season: 
            name_temp = 'averages_'+season 
        dfs[name]['Player'] = dfs[name]['Player'].apply(lambda x: clean_player(x))
        dfs[name].to_sql(name_temp, con = engine, if_exists = 'replace', index = False)  
        
    master.to_sql('master_'+season, con=engine, if_exists='replace', index=False) 
     